## Домашнє завдання до модуля «Алгоритми навчання з вчителем Ч.2»

### 1. Здійсніть імпорт необхідних пакетів.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


 ### 2. Завантажте набір даних Rain in Australia

In [2]:

df = pd.read_csv('rain.csv')
print(df.head())


         Date Location  MinTemp  ...  Temp3pm  RainToday  RainTomorrow
0  2008-12-01   Albury     13.4  ...     21.8         No            No
1  2008-12-02   Albury      7.4  ...     24.3         No            No
2  2008-12-03   Albury     12.9  ...     23.2         No            No
3  2008-12-04   Albury      9.2  ...     26.5         No            No
4  2008-12-05   Albury     17.5  ...     29.7         No            No

[5 rows x 23 columns]


In [3]:
# Підрахунок пропущених значень
missing_values = df.isnull().sum()

# Відображення кількості пропущених значень у кожному стовпці
print(missing_values)

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


### 3: Видалення ознак з великою кількістю пропущених значень

In [4]:
# Видалення ознак з понад 30% пропущених значень
threshold = 0.3  # 30% пропущених значень
columns_to_keep = missing_values[missing_values / len(df) <= threshold].index

# Фільтрація датасету
df = df[columns_to_keep]

# Перевіримо результат
print("Оновлений набір даних:\n", df.head())

Оновлений набір даних:
          Date Location  MinTemp  ...  Temp3pm  RainToday RainTomorrow
0  2008-12-01   Albury     13.4  ...     21.8         No           No
1  2008-12-02   Albury      7.4  ...     24.3         No           No
2  2008-12-03   Albury     12.9  ...     23.2         No           No
3  2008-12-04   Albury      9.2  ...     26.5         No           No
4  2008-12-05   Albury     17.5  ...     29.7         No           No

[5 rows x 19 columns]


### 3.2. Створіть підмножини набору даних із числовими та категоріальними ознаками.

In [5]:
# Перевірка типів даних
print(df.dtypes)


Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object


In [6]:
# Вибір числових ознак
numeric_features = df.select_dtypes(include=['int64', 'float64'])

# Вибір категоріальних ознак
categorical_features = df.select_dtypes(include=['object'])

# Перевіримо підмножини
print("Числові ознаки:\n", numeric_features.head())
print("Категоріальні ознаки:\n", categorical_features.head())


Числові ознаки:
    MinTemp  MaxTemp  Rainfall  ...  Pressure3pm  Temp9am  Temp3pm
0     13.4     22.9       0.6  ...       1007.1     16.9     21.8
1      7.4     25.1       0.0  ...       1007.8     17.2     24.3
2     12.9     25.7       0.0  ...       1008.7     21.0     23.2
3      9.2     28.0       0.0  ...       1012.8     18.1     26.5
4     17.5     32.3       1.0  ...       1006.0     17.8     29.7

[5 rows x 12 columns]
Категоріальні ознаки:
          Date Location WindGustDir WindDir9am WindDir3pm RainToday RainTomorrow
0  2008-12-01   Albury           W          W        WNW        No           No
1  2008-12-02   Albury         WNW        NNW        WSW        No           No
2  2008-12-03   Albury         WSW          W        WSW        No           No
3  2008-12-04   Albury          NE         SE          E        No           No
4  2008-12-05   Albury           W        ENE         NW        No           No


In [7]:
# Збереження підмножин
numeric_data = numeric_features
categorical_data = categorical_features

### 3.3. Змініть тип колонки Date на тип datetimeі створіть додаткові колонки Year та Month.

In [8]:
# Зміна типу колонки Date на datetime
df['Date'] = pd.to_datetime(df['Date'])

# Перевірка змін
print(df['Date'].dtypes)

datetime64[ns]


In [10]:
# Створення нових колонок Year і Month
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Перевірка результату
print(df[['Date', 'Year', 'Month']].head())


        Date  Year  Month
0 2008-12-01  2008     12
1 2008-12-02  2008     12
2 2008-12-03  2008     12
3 2008-12-04  2008     12
4 2008-12-05  2008     12


###  3.4. Переміcтить створену нову колонку Year з підмножини набору із категоріальними ознаками до підмножини із числовими ознаками.

In [11]:
# Видалення Year з категоріальних ознак
if 'Year' in categorical_data.columns:
    categorical_data = categorical_data.drop(columns=['Year'])


In [12]:
# Додавання Year до числових ознак
numeric_data['Year'] = df['Year']


In [13]:
# Перевірка оновлених підмножин
print("Оновлені числові ознаки:\n", numeric_data.head())
print("Оновлені категоріальні ознаки:\n", categorical_data.head())


Оновлені числові ознаки:
    MinTemp  MaxTemp  Rainfall  ...  Temp9am  Temp3pm  Year
0     13.4     22.9       0.6  ...     16.9     21.8  2008
1      7.4     25.1       0.0  ...     17.2     24.3  2008
2     12.9     25.7       0.0  ...     21.0     23.2  2008
3      9.2     28.0       0.0  ...     18.1     26.5  2008
4     17.5     32.3       1.0  ...     17.8     29.7  2008

[5 rows x 13 columns]
Оновлені категоріальні ознаки:
          Date Location WindGustDir WindDir9am WindDir3pm RainToday RainTomorrow
0  2008-12-01   Albury           W          W        WNW        No           No
1  2008-12-02   Albury         WNW        NNW        WSW        No           No
2  2008-12-03   Albury         WSW          W        WSW        No           No
3  2008-12-04   Albury          NE         SE          E        No           No
4  2008-12-05   Albury           W        ENE         NW        No           No


### 3.5. Розбийте підмножини на тренувальну і тестову вибірки за такою логікою: до тестової вибірки віднесіть всі об'єкти із набору даних із останнім (максимальним) роком спостережень, а для навчання моделі залиште всі інші об'єкти.

In [14]:
# Знаходимо максимальний рік
max_year = df['Year'].max()
print(f"Максимальний рік спостережень: {max_year}")


Максимальний рік спостережень: 2017


In [15]:
# Розділення даних
train_data = df[df['Year'] < max_year]
test_data = df[df['Year'] == max_year]

# Перевірка розмірів вибірок
print(f"Розмір тренувальної вибірки: {len(train_data)}")
print(f"Розмір тестової вибірки: {len(test_data)}")


Розмір тренувальної вибірки: 136837
Розмір тестової вибірки: 8623


In [16]:
# Перевірка унікальних значень в колонці Year
print("Унікальні роки в тренувальній вибірці:", train_data['Year'].unique())
print("Унікальні роки в тестовій вибірці:", test_data['Year'].unique())


Унікальні роки в тренувальній вибірці: [2008 2009 2010 2011 2012 2013 2014 2015 2016 2007]
Унікальні роки в тестовій вибірці: [2017]


### 4. Відновіть пропущені дані за допомогою об'єкта SimpleImputer з пакету sklearn.

In [17]:
from sklearn.impute import SimpleImputer

In [18]:
# Імп'ютер для числових даних (стратегія: середнє значення)
numeric_imputer = SimpleImputer(strategy='mean')

# Застосування імп'ютера до числових ознак
numeric_data_imputed = pd.DataFrame(numeric_imputer.fit_transform(numeric_data), columns=numeric_data.columns)

# Перевірка результату
print(numeric_data_imputed.head())


   MinTemp  MaxTemp  Rainfall  ...  Temp9am  Temp3pm    Year
0     13.4     22.9       0.6  ...     16.9     21.8  2008.0
1      7.4     25.1       0.0  ...     17.2     24.3  2008.0
2     12.9     25.7       0.0  ...     21.0     23.2  2008.0
3      9.2     28.0       0.0  ...     18.1     26.5  2008.0
4     17.5     32.3       1.0  ...     17.8     29.7  2008.0

[5 rows x 13 columns]


In [19]:
# Імп'ютер для категоріальних даних (стратегія: найчастіше значення)
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Застосування імп'ютера до категоріальних ознак
categorical_data_imputed = pd.DataFrame(categorical_imputer.fit_transform(categorical_data), columns=categorical_data.columns)

# Перевірка результату
print(categorical_data_imputed.head())


         Date Location WindGustDir WindDir9am WindDir3pm RainToday RainTomorrow
0  2008-12-01   Albury           W          W        WNW        No           No
1  2008-12-02   Albury         WNW        NNW        WSW        No           No
2  2008-12-03   Albury         WSW          W        WSW        No           No
3  2008-12-04   Albury          NE         SE          E        No           No
4  2008-12-05   Albury           W        ENE         NW        No           No


### 5. Нормалізуйте числові ознаки за допомогою об'єкта StandardScaler з пакету sklearn.

In [20]:
from sklearn.preprocessing import StandardScaler



In [21]:
# Ініціалізація StandardScaler
scaler = StandardScaler()

# Нормалізація числових даних
numeric_data_scaled = pd.DataFrame(scaler.fit_transform(numeric_data_imputed), columns=numeric_data_imputed.columns)

# Перевіримо результат
print(numeric_data_scaled.head())


    MinTemp   MaxTemp  Rainfall  ...   Temp9am   Temp3pm      Year
0  0.189447 -0.045336 -0.210072  ... -0.014053  0.017023 -1.879575
1 -0.753101  0.265043 -0.281650  ...  0.032464  0.381985 -1.879575
2  0.110901  0.349692 -0.281650  ...  0.621685  0.221402 -1.879575
3 -0.470337  0.674180 -0.281650  ...  0.172017  0.703151 -1.879575
4  0.833521  1.280830 -0.162353  ...  0.125499  1.170302 -1.879575

[5 rows x 13 columns]


In [22]:
print("Середнє значення після нормалізації:\n", numeric_data_scaled.mean())
print("Стандартне відхилення після нормалізації:\n", numeric_data_scaled.std())


Середнє значення після нормалізації:
 MinTemp         -3.751525e-16
MaxTemp         -6.377592e-16
Rainfall         8.753558e-17
WindGustSpeed   -1.647154e-16
WindSpeed9am     3.415451e-16
WindSpeed3pm     1.617845e-16
Humidity9am     -2.438491e-16
Humidity3pm      6.565169e-17
Pressure9am     -1.209867e-14
Pressure3pm     -2.608560e-14
Temp9am          3.126271e-17
Temp3pm          3.751525e-17
Year             2.080221e-14
dtype: float64
Стандартне відхилення після нормалізації:
 MinTemp          1.000003
MaxTemp          1.000003
Rainfall         1.000003
WindGustSpeed    1.000003
WindSpeed9am     1.000003
WindSpeed3pm     1.000003
Humidity9am      1.000003
Humidity3pm      1.000003
Pressure9am      1.000003
Pressure3pm      1.000003
Temp9am          1.000003
Temp3pm          1.000003
Year             1.000003
dtype: float64


### 6. Виконайте кодування категоріальних ознак за допомогою об’єкта OneHotEncoder з пакету sklearn.

In [23]:
from sklearn.preprocessing import OneHotEncoder


In [24]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


In [25]:
categorical_data_encoded = pd.DataFrame(
    encoder.fit_transform(categorical_data_imputed),
    columns=encoder.get_feature_names_out(categorical_data_imputed.columns)
)


In [26]:
# Перевірка перших рядків закодованих даних
print(categorical_data_encoded.head())

# Перевірка кількості колонок після кодування
print(f"Кількість колонок після кодування: {categorical_data_encoded.shape[1]}")


   Date_2007-11-01  Date_2007-11-02  ...  RainTomorrow_No  RainTomorrow_Yes
0              0.0              0.0  ...              1.0               0.0
1              0.0              0.0  ...              1.0               0.0
2              0.0              0.0  ...              1.0               0.0
3              0.0              0.0  ...              1.0               0.0
4              0.0              0.0  ...              1.0               0.0

[5 rows x 3537 columns]
Кількість колонок після кодування: 3537


In [27]:
# Виключення колонки Date
categorical_data_cleaned = categorical_data_imputed.drop(columns=['Date'])


In [28]:
# Застосування OneHotEncoder
categorical_data_encoded = pd.DataFrame(
    encoder.fit_transform(categorical_data_cleaned),
    columns=encoder.get_feature_names_out(categorical_data_cleaned.columns)
)

# Перевірка результату
print(categorical_data_encoded.head())
print(f"Кількість колонок після кодування: {categorical_data_encoded.shape[1]}")


   Location_Adelaide  Location_Albany  ...  RainTomorrow_No  RainTomorrow_Yes
0                0.0              0.0  ...              1.0               0.0
1                0.0              0.0  ...              1.0               0.0
2                0.0              0.0  ...              1.0               0.0
3                0.0              0.0  ...              1.0               0.0
4                0.0              0.0  ...              1.0               0.0

[5 rows x 101 columns]
Кількість колонок після кодування: 101


In [29]:
# Додавання колонки Date назад
final_data = pd.concat([categorical_data_encoded, categorical_data_imputed['Date']], axis=1)

# Перевірка фінального набору даних
print(final_data.head())


   Location_Adelaide  Location_Albany  ...  RainTomorrow_Yes        Date
0                0.0              0.0  ...               0.0  2008-12-01
1                0.0              0.0  ...               0.0  2008-12-02
2                0.0              0.0  ...               0.0  2008-12-03
3                0.0              0.0  ...               0.0  2008-12-04
4                0.0              0.0  ...               0.0  2008-12-05

[5 rows x 102 columns]


### 7. Об'єднайте підмножини з числовими і категоріальними ознаками (після кодування) для побудови моделі за допомогою об’єкта LogisticRegression з пакету sklearn.

In [30]:
# Об'єднання числових і категоріальних даних
final_data = pd.concat([numeric_data_scaled, categorical_data_encoded], axis=1)

# Перевіримо об'єднаний набір даних
print(final_data.head())
print(f"Розмір об'єднаного набору даних: {final_data.shape}")


    MinTemp   MaxTemp  ...  RainTomorrow_No  RainTomorrow_Yes
0  0.189447 -0.045336  ...              1.0               0.0
1 -0.753101  0.265043  ...              1.0               0.0
2  0.110901  0.349692  ...              1.0               0.0
3 -0.470337  0.674180  ...              1.0               0.0
4  0.833521  1.280830  ...              1.0               0.0

[5 rows x 114 columns]
Розмір об'єднаного набору даних: (145460, 114)


In [31]:
# Визначення ознак (X) і цільової змінної (y)
X = final_data.drop(columns=['RainTomorrow_Yes', 'RainTomorrow_No'])
y = categorical_data_encoded['RainTomorrow_Yes']  # Цільова змінна

# Перевіримо розміри X і y
print(f"Розмір X: {X.shape}")
print(f"Розмір y: {y.shape}")


Розмір X: (145460, 112)
Розмір y: (145460,)


In [32]:
from sklearn.model_selection import train_test_split

# Розбиття на тренувальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Перевіримо розміри вибірок
print(f"Розмір тренувальної вибірки: {X_train.shape}")
print(f"Розмір тестової вибірки: {X_test.shape}")


Розмір тренувальної вибірки: (116368, 112)
Розмір тестової вибірки: (29092, 112)


In [33]:
from sklearn.linear_model import LogisticRegression

# Ініціалізація моделі
model = LogisticRegression(solver='liblinear', max_iter=1000)

# Навчання моделі
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, solver='liblinear')

In [34]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Передбачення
y_pred = model.predict(X_test)

# Метрики
print("Матриця плутанини:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Точність:", accuracy_score(y_test, y_pred))


Матриця плутанини:
 [[21544  1128]
 [ 3386  3034]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.86      0.95      0.91     22672
         1.0       0.73      0.47      0.57      6420

    accuracy                           0.84     29092
   macro avg       0.80      0.71      0.74     29092
weighted avg       0.83      0.84      0.83     29092

Точність: 0.8448370686099271


In [35]:
import time

solvers = ['lbfgs', 'saga', 'newton-cg']

for solver in solvers:
    print(f"\nМодель з solver='{solver}':")
    try:
        start_time = time.time()
        
        # Ініціалізація моделі
        model = LogisticRegression(solver=solver, max_iter=1000)
        
        # Навчання моделі
        model.fit(X_train, y_train)
        
        # Передбачення
        y_pred = model.predict(X_test)
        
        # Виведення точності
        print("Точність:", accuracy_score(y_test, y_pred))
        print("Час виконання:", time.time() - start_time, "секунд")
    except Exception as e:
        print(f"Помилка при використанні solver='{solver}': {e}")



Модель з solver='lbfgs':
Точність: 0.8447683211879554
Час виконання: 0.9252820014953613 секунд

Модель з solver='saga':


C:\ProgramData\anaconda3\envs\mlf\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Точність: 0.8448370686099271
Час виконання: 148.24224495887756 секунд

Модель з solver='newton-cg':
Точність: 0.8448714423209129
Час виконання: 1.3993537425994873 секунд


### Порівняння метрик:
1. Точність (Accuracy):

    - У новій моделі точність складає 84.4%, що є покращенням порівняно з базовою моделлю, яка мала 80%.
    - Це свідчить про кращу здатність моделі правильно класифікувати дані.
2. F1-score:

 - Для класу "No" (основний клас): 0.91, що значно вище за базову модель.
 - Для класу "Yes" (менш поширений клас): 0.57, що трохи нижче очікуваного. Однак це є типовим для задачі з дисбалансом класів.

3. Recall та Precision:

    - Precision: значно покращився для основного класу, що означає, що менше хибнопозитивних результатів.
    - Recall: для менш поширеного класу "Yes" залишився майже таким самим, як у базовій моделі.

### Загальний висновок:
Нова модель із solver 'lbfgs' показала кращі результати, особливо для основного класу "No". Це досягнуто завдяки оптимізації гіперпараметрів. Проте, для класу "Yes" показники Recall потребують покращення, що може бути досягнуто через:

    - Балансування вибірки (oversampling/undersampling).
    - Використання ваг класів (class_weight='balanced').
    - Застосування методів обробки дисбалансу класів.

### 8. Розрахуйте метрики нової моделі за допомогою методу classification_report() з пакета sklearn